In [1]:
import os
import numpy as np
import tensorflow as tf

C:\Users\ketul\anaconda3\envs\TensorflowGPU\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
for dirpath, dirnames, filenames in os.walk("chest_xray"):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 3 directories and 0 images in 'chest_xray'.
There are 2 directories and 0 images in 'chest_xray\test'.
There are 0 directories and 234 images in 'chest_xray\test\NORMAL'.
There are 0 directories and 390 images in 'chest_xray\test\PNEUMONIA'.
There are 2 directories and 0 images in 'chest_xray\train'.
There are 0 directories and 1341 images in 'chest_xray\train\NORMAL'.
There are 0 directories and 3875 images in 'chest_xray\train\PNEUMONIA'.
There are 2 directories and 0 images in 'chest_xray\val'.
There are 0 directories and 8 images in 'chest_xray\val\NORMAL'.
There are 0 directories and 8 images in 'chest_xray\val\PNEUMONIA'.


In [2]:
num_of_normal_images = len(os.listdir("chest_xray/train/NORMAL"))
num_of_normal_images

1341

In [3]:
num_of_normal_images = len(os.listdir("chest_xray/train/PNEUMONIA"))
num_of_normal_images

3875

In [4]:
num_of_normal_images = len(os.listdir("chest_xray/test/NORMAL"))
num_of_normal_images

234

In [5]:
num_of_normal_images = len(os.listdir("chest_xray/test/PNEUMONIA"))
num_of_normal_images

390

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

tf.random.set_seed(42)

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_dir = "chest_xray/train/"
test_dir = "chest_xray/test/"


train_data = train_datagen.flow_from_directory(train_dir,
                                               batch_size=64, # number of images to process at a time 
                                               target_size=(224, 224), # convert all images to be 224 x 224
                                               class_mode="binary", # type of problem we're working on
                                               seed=42)

valid_data = test_datagen.flow_from_directory(test_dir,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="binary",
                                               seed=42)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [15]:
model_1 = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(filters=10, 
                         kernel_size=3, # can also be (3, 3)
                         activation="relu", 
                         input_shape=(224, 224, 3)), # first layer specifies input shape (height, width, colour channels)
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=2, # pool_size can also be (2, 2)
                            padding="valid"), # padding can also be 'same'
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.Conv2D(10, 3, activation="relu"), # activation='relu' == tf.keras.layers.Activations(tf.nn.relu)
  tf.keras.layers.MaxPool2D(2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(1, activation="sigmoid") # binary activation output
])

# Compile the model
model_1.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

# Fit the model
history_1 = model_1.fit(train_data,
                        epochs=5,
                        steps_per_epoch=len(train_data),
                        validation_data=valid_data,
                        validation_steps=len(valid_data))

Epoch 1/5
82/82 [==============================] - 41s 501ms/step - loss: 0.3212 - accuracy: 0.8614 - val_loss: 0.6217 - val_accuracy: 0.7484
Epoch 2/5
82/82 [==============================] - 42s 511ms/step - loss: 0.1333 - accuracy: 0.9500 - val_loss: 0.9057 - val_accuracy: 0.7516
Epoch 3/5
82/82 [==============================] - 41s 503ms/step - loss: 0.0974 - accuracy: 0.9672 - val_loss: 0.9750 - val_accuracy: 0.7404
Epoch 4/5
82/82 [==============================] - 41s 504ms/step - loss: 0.1054 - accuracy: 0.9632 - val_loss: 0.8785 - val_accuracy: 0.7500
Epoch 5/5
82/82 [==============================] - 43s 523ms/step - loss: 0.0785 - accuracy: 0.9726 - val_loss: 1.0927 - val_accuracy: 0.7356


In [10]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4900507692812608574
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2254123828
locality {
  bus_id: 1
  links {
  }
}
incarnation: 308533925223448196
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650 Ti with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]
